In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import struct

In [2]:
FNAME = '/home/david/projects/interferometry/data/SB320-202012281240-lba_outer.vis'

MAGIC = '0x000000003B98F002'
int(MAGIC, 0)

999878658

In [3]:
NANT = 576
SUB = 320
NCHAN = 1
POL2REC = 4
LEN_HDR = 512
NBLINE = NANT * (NANT + 1) // 2
PAYLOAD = NBLINE * NCHAN * POL2REC * 8
RECSIZE = PAYLOAD + LEN_HDR

fsize = os.path.getsize(FNAME)
nrec = fsize // RECSIZE
nrec

418

In [4]:
def read_vis(f):
    s = f.read(RECSIZE)

#     header = s[:512]
    data = s[512:]

    magic, = struct.unpack("<Q", s[0:8])
    magic = np.uint32(magic)
    assert int(magic) == int(MAGIC, 0)

    data = np.reshape(np.frombuffer(data, dtype=np.complex64), (NBLINE, NCHAN, POL2REC))
    return data

In [5]:
# xx, xy, yx, yy
with open(FNAME, 'rb') as f:

    f.seek(0 * RECSIZE)

    vis = read_vis(f)
data = vis.copy()
data.shape

(166176, 1, 4)

In [6]:
from parse_lba_antennafield import parse_lba_antennafield
L = parse_lba_antennafield('../antennafields/a12-AntennaField.conf', [SUB])

from scipy.constants import c
meters_to_wavelengths = 2 * SUB * 2e8 / (1024 * c)   # frequency over speed is wavelength
L_2d = L[:, :2]
distances = meters_to_wavelengths * (L_2d[:, None] - L_2d)

In [7]:
res = 2300
dx = 1.0 / res
out_ax = [(dx, res), (dx, res)]

L = np.linspace(-1, 1, res)
M = np.linspace(-1, 1, res)
mask = np.ones((res, res))
xv, yv = np.meshgrid(L, M)

u, v = distances[np.tril_indices(576)].T

In [8]:
image = data.copy()

im = image.mean(axis=(-1, -2))

In [9]:
def get_beta(W, alpha):

    # see Beatty et al. (2005)
    beta = np.pi*np.sqrt((W*W/alpha/alpha)*(alpha - 0.5)*(alpha - 0.5) - 0.8)

    return beta


In [10]:
# def grid_2d(u, v, vis, du, Nu, umin, dv, Nv, vmin, alpha, W, hflag_u, hflag_v):
#     nvis = len(u)
    
#     tv1 = np.zeros(W)
#     tu1 = np.zeros(W)
#     tvis1 = np.zeros(W)
    
#     beta = get_beta(W, alpha)

#     for i in range(nvis):
        
#         su = u[i]
#         sv = v[i]
        
#         svis = vis[i]
        
#         grid_1d_from_2d(su, svis, du, W, beta, sv, tu1, tvis1, tv1)
        

In [11]:
def pre_grid(out_ax, alpha=1.5):
    dx = out_ax[0][0]
    Nx = out_ax[0][1]
    xmin = 0
    du = 1 / dx / Nx / alpha
    Nu = int(alpha * Nx)
    umin = 0

    dy = out_ax[1][0]
    Ny = out_ax[1][1]
    ymin = 0
    dv = 1 / dx / Nx / alpha
    Nv = int(alpha * Ny)
    vmin = 0
    
    do_preshift = True
    if do_preshift:
        vmin = -0.5 * Nv * dv
        umin = -0.5 * Nu * du
            
    do_postshift = True
    if do_postshift:
        xmin = -0.5 * Nx * dx
        ymin = -0.5 * Ny * dy
        
    print(xmin, ymin, Ny, dy)
    
    return du, Nu, umin, dv, Nv, vmin
        
    
    

In [2]:
import numpy as np

In [12]:
alpha = 1.5
W = 6
du, Nu, umin, dv, Nv, vmin = pre_grid(out_ax, alpha)

-0.5 -0.5 2300 0.0004347826086956522


In [13]:
from gridding import grid_2d

In [14]:
inp_grid = grid_2d(u, v, im, du, Nu, umin, dv, Nv, vmin, alpha, W, True, True)

  0%|          | 4/5982336 [00:00<50:28:14, 32.93it/s]

0
sumstart 0j
1722.5 1722
1722.5 1722
gvvis
0j
(15.369683064791577+0j)
True
summid (15.369683064791577+0j)
gvmid
(15.369683064791577+0j)
hflag_u 1728.5 1728
hflag_v 1728.5 1728
gvis
0j
(15.369683064791577-0j)
True
here 0j (15.369683064791577-0j) (15.369683064791577+0j)
(15.369683064791577+0j)
(30.739366129583154+0j)
(15.369683064791577+0j)
gv
(15.369683064791577+0j)
summend (30.739366129583154+0j)
(30.739366129583154+0j)
1
sumstart (30.739366129583154+0j)
1722.5 1722
1723.5000000000002 1723
gvvis
0j
(18987.193323824475+0j)
True
summid (19017.93268995406+0j)
gvmid
(18987.193323824475+0j)
hflag_u 1728.5 1728
hflag_v 1727.5 1727
gvis
0j
(18987.193323824475-0j)
True
here 0j (18987.193323824475-0j) (18987.193323824475+0j)
(19017.93268995406+0j)
(38005.126013778536+0j)
(18987.193323824475+0j)
gv
(18987.193323824475+0j)
summend (38005.126013778536+0j)
(38005.126013778536+0j)
2
sumstart (38005.126013778536+0j)
1722.5 1722
1724.5 1724
gvvis
0j
(188953.40529532547+0j)
True
summid (226958.5313091

  0%|          | 12/5982336 [00:00<52:41:21, 31.54it/s]

summid (25064697.759096455+0j)
gvmid
(23456144.723123055+0j)
hflag_u 1727.5 1727
hflag_v 1727.5 1727
gvis
0j
(23456144.723123055-0j)
True
here 0j (23456144.723123055-0j) (23456144.723123055+0j)
(25064697.759096455+0j)
(48520842.48221951+0j)
(23456144.723123055+0j)
gv
(23456144.723123055+0j)
summend (48520842.48221951+0j)
(48520842.48221951+0j)
8
sumstart (48520842.48221951+0j)
1723.5000000000002 1723
1724.5 1724
gvvis
0j
(233426728.47665226+0j)
True
summid (281947570.9588717+0j)
gvmid
(233426728.47665226+0j)
hflag_u 1727.5 1727
hflag_v 1726.5000000000002 1726
gvis
0j
(233426728.47665226-0j)
True
here 0j (233426728.47665226-0j) (233426728.47665226+0j)
(281947570.9588717+0j)
(515374299.435524+0j)
(233426728.47665226+0j)
gv
(233426728.47665226+0j)
summend (515374299.435524+0j)
(515374299.435524+0j)
9
sumstart (515374299.435524+0j)
1723.5000000000002 1723
1725.5 1725
gvvis
0j
(456335480.3148535+0j)
True
summid (971709779.7503777+0j)
gvmid
(456335480.3148535+0j)
hflag_u 1727.5 1727
hflag_v 

  0%|          | 21/5982336 [00:00<45:21:13, 36.64it/s]

...

  0%|          | 43/5982336 [00:01<45:53:26, 36.21it/s]

(80249210851.66347+0j)
38
sumstart (80249210851.66347+0j)
1721.5 1721
1726.5000000000002 1726
gvvis
0j
(4554403.216422962+4874698.869451369j)
True
summid (80253765254.8799+4874698.869451368j)
gvmid
(4554403.216422962+4874698.869451369j)
hflag_u 1729.5000000000002 1729
hflag_v 1724.5 1724
gvis
0j
(4554403.216422962-4874698.869451369j)
True
here 0j (4554403.216422962-4874698.869451369j) (4554403.216422962-4874698.869451369j)
(80253765254.8799+4874698.869451368j)
(80258319658.09631-9.313225746154785e-10j)
(4554403.216422962-4874698.869451369j)
gv
(4554403.216422962-4874698.869451369j)
summend (80258319658.09631-9.313225746154785e-10j)
(80258319658.09631-9.313225746154785e-10j)
39
sumstart (80258319658.09631-9.313225746154785e-10j)
1721.5 1721
1727.5 1727
gvvis
0j
(3664614.979438214+3922334.8588955975j)
True
summid (80261984273.07574+3922334.858895597j)
gvmid
(3664614.979438214+3922334.8588955975j)
hflag_u 1729.5000000000002 1729
hflag_v 1723.5000000000002 1723
gvis
0j
(3664614.979438214-3

  0%|          | 45/5982336 [00:01<46:08:16, 36.02it/s]

summend (80960426367.79738+8.940696716308594e-08j)
(80960426367.79738+8.940696716308594e-08j)
45
sumstart (80960426367.79738+8.940696716308594e-08j)
1722.5 1722
1727.5 1727
gvvis
(18987.19332382454+0j)
(209641166.24962133+224384514.84103057j)
True
summid (81170067534.04701+224384514.8410307j)
gvmid
(209660153.44294515+224384514.84103057j)
hflag_u 1728.5 1728
hflag_v 1723.5000000000002 1723
gvis
(18987.19332382454+0j)
(209641166.24962133-224384514.84103057j)
True
here (18987.19332382454+0j) (209641166.24962133-224384514.84103057j) (209660153.44294515-224384514.84103057j)
(81170067534.04701+224384514.8410307j)
(81379708700.29663+1.4901161193847656e-07j)
(209660153.44294515-224384514.84103057j)
gv
(209660153.44294515-224384514.84103057j)
summend (81379708700.29663+1.4901161193847656e-07j)
(81379708700.29663+1.4901161193847656e-07j)


RuntimeError: No active exception to reraise

In [ ]:
# Complex sum wrong...

inp_grid.sum()

In [ ]:
preshift_axes = None
inp_grid = np.fft.fftshift(inp_grid, axes=None)


In [ ]:
fftaxes = None
out = np.fft.fftn(inp_grid, axes=fftaxes)


In [ ]:
postshift_axes = None
out = np.fft.fftshift(out, axes=postshift_axes)


In [ ]:
dx = out_ax[0][0]
Nx = out_ax[0][1]

dy = out_ax[0][1]
Ny = out_ax[0][1]

In [ ]:
tndxx = int(0.5*Nx*(alpha-1))
tndxy = int(0.5*Ny*(alpha-1))
xl = 0
yl = 0

do_postshift = True

if do_postshift:
    if postshift_axes == None:
        xl = tndxx
        yl = tndxy
    else:
        if postshift_axes.count(0)>0:
            xl = tndxx
        if postshift_axes.count(1)>0:
            yl = tndxy

out = out[xl:xl+Nx, yl:yl+Ny]


In [ ]:
from gridding import get_grid_corr_2d

In [ ]:
xmin = -0.5*Nx*dx
ymin = -0.5*Ny*dy


In [ ]:
gc = get_grid_corr_2d(dx, Nx, xmin, dy, Ny, ymin, \
    du, dv, W, alpha)


In [ ]:
final = out / c

In [ ]:
final.shape

In [ ]:
im = np.rot90(np.real(final))

In [ ]:
plt.imshow(im)